In [16]:
import tensorflow as tf
import numpy as np
#from utils import build_model
import pandas as pd
from tqdm import tqdm
import imageio
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, Dropout, Flatten, Dense
from keras.regularizers import l2
import statsmodels.api as sm
from utils import build_model, find_closest_element, Fy
from scipy.stats import norm

In [5]:
checkpoint_path = '../../../data/models/copula_cpl/export/'

In [6]:
tf.train.latest_checkpoint(checkpoint_path)

'../../../data/models/copula_cpl/export/model.ckpt-74000'

In [7]:
model = build_model()
model.load_weights(tf.train.latest_checkpoint(checkpoint_path))

In [9]:
path_all_imgs = '../../../data/commaai/training_files_filtered/indices/val_indices.csv'
all_img_df = pd.read_csv(path_all_imgs)
img_path_base = '../../../data/commaai/train_bags_2/'
density_path = '../../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

In [10]:
# extract Bzetas by reading in images and predicting
labels = []
preds = []
for i in tqdm(range(0,all_img_df.shape[0])): 
    img = imageio.imread(str(img_path_base + all_img_df.loc[i,'path']))[:,:,0:3]/255
    pred = model.predict(img.reshape(1,66,200,3))
    label = all_img_df.loc[i,'angle']
    labels.append(label)       
    preds.append(pred)

# rearrange to arrays
labels = np.array(labels)
preds = np.array(preds)

  0%|          | 4/10472 [00:02<2:04:58,  1.40it/s]

100%|██████████| 10472/10472 [45:40<00:00,  3.82it/s] 


In [18]:
y_pred = np.array([density.loc[find_closest_element(norm.cdf(i), density['cdf']), 'axes'] for i in tqdm(preds.reshape(-1))])




  0%|          | 0/10472 [00:35<?, ?it/s]



  2%|▏         | 214/10472 [00:00<00:04, 2136.09it/s]


  4%|▍         | 453/10472 [00:00<00:04, 2205.08it/s]


  7%|▋         | 694/10472 [00:00<00:04, 2262.72it/s]


  9%|▉         | 931/10472 [00:00<00:04, 2292.46it/s]


 11%|█         | 1147/10472 [00:00<00:04, 2248.63it/s]


 13%|█▎        | 1337/10472 [00:00<00:04, 2125.08it/s]


 15%|█▍        | 1527/10472 [00:00<00:04, 1908.97it/s]


 16%|█▋        | 1715/10472 [00:00<00:04, 1899.02it/s]


 18%|█▊        | 1897/10472 [00:00<00:04, 1729.13it/s]


 20%|█▉        | 2067/10472 [00:01<00:05, 1619.37it/s]


 22%|██▏       | 2260/10472 [00:01<00:04, 1700.61it/s]


 24%|██▎       | 2482/10472 [00:01<00:04, 1828.97it/s]


 26%|██▌       | 2702/10472 [00:01<00:04, 1925.58it/s]


 28%|██▊       | 2949/10472 [00:01<00:03, 2060.70it/s]


 30%|███       | 3161/10472 [00:01<00:03, 2054.57it/s]


 32%|███▏      | 3385/10472 [00:01<00:03, 2105.49it/s]


 34%|███▍      | 3599/10472 [00:01<00:03, 20

In [19]:
np.mean(np.abs(labels - y_pred)**2)

7.333487690026303